In [23]:
import requests
from fastapi import FastAPI
from pydantic import BaseModel
from nest_asyncio import apply
import uvicorn

from typing import Any
from fastapi.responses import RedirectResponse

In [31]:
from fastapi.encoders import jsonable_encoder
fake_db = {}

In [37]:
app = FastAPI()

class Item(BaseModel):
    title: str
    description: str | None = None


app = FastAPI()
@app.post("/items/{id}")
def update_item(id: str, item: Item):
    json_compatible_item_data = jsonable_encoder(item)
    fake_db[id] = json_compatible_item_data
    
    return JSONResponse(content={'message':'database updated'})


if __name__ == "__main__":
    apply()
    uvicorn.run(app,port=4444)

INFO:     Started server process [102663]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:39326 - "POST /items/1 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [102663]


In [34]:
data = {'title':'test','description':'this is a test'}
res = requests.post("http://127.0.0.1:4444/items/1",json=data)
res.content.decode('utf-8')

'{"message":"database updated"}'